In [ ]:
import pandas as pd
import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/final_prj data/twitter_new.csv',encoding='latin-1')

In [ ]:
df.shape

(1599999, 6)

In [ ]:
df.rename(columns={"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":'Tweet','_TheSpecialOne_':'Author','0':'Polarity'},inplace=True)

In [ ]:
df

,Polarity,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,Author,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Polarity                      1599999 non-null  int64 
 1   1467810369                    1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009  1599999 non-null  object
 3   NO_QUERY                      1599999 non-null  object
 4   Author                        1599999 non-null  object
 5   Tweet                         1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
dff=df[['Tweet','Polarity']]

In [ ]:
dff['Tweet']=dff['Tweet'].str.lower()

In [ ]:
dff['Polarity'].unique()

array([0, 4])

In [ ]:
dff

,Tweet,Polarity
0,is upset that he can't update his facebook by ...,0
1,@kenichan i dived many times for the ball. man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@kwesidei not the whole crew,0
...,...,...
1599994,just woke up. having no school is the best fee...,4
1599995,thewdb.com - very cool to hear old walt interv...,4
1599996,are you ready for your mojo makeover? ask me f...,4
1599997,happy 38th birthday to my boo of alll time!!! ...,4


In [ ]:
stopwords=set(stopwords.words('english'))

In [ ]:
#Removing stopwords
def clean(text):
  words=text.split()
  words=[word for word in words if word.lower() not in stopwords]
  return ' '.join(words)

In [ ]:
dff['Tweet']=dff['Tweet'].apply(clean)

In [ ]:
dff

,Tweet,Polarity
0,upset can't update facebook texting it... migh...,0
1,@kenichan dived many times ball. managed save ...,0
2,whole body feels itchy like fire,0
3,"@nationwideclass no, behaving all. i'm mad. he...",0
4,@kwesidei whole crew,0
...,...,...
1599994,woke up. school best feeling ever,4
1599995,thewdb.com - cool hear old walt interviews! â...,4
1599996,ready mojo makeover? ask details,4
1599997,happy 38th birthday boo alll time!!! tupac ama...,4


In [ ]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations

In [ ]:
punctuations_list

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Removing punctuation
def clean_punc(text):
  return ''.join(x for x in text if x not in punctuations_list)

In [ ]:
dff['Tweet']=dff['Tweet'].apply(lambda x:clean_punc(x))

In [ ]:
def extra_clean(text):
  c1=re.sub(r'(.)1+', r'1', text)                         #removing repeating characters
  c2=re.sub('((www.[^s]+)|(https?://[^s]+))',' ',c1)      #removing URLs
  c3=re.sub('[0-9]+', '',c2)                              #removing numeric numbers
  return c3

In [ ]:
dff['Tweet']=dff['Tweet'].apply(lambda x:extra_clean(x))

In [ ]:
dff

,Tweet,Polarity
0,upset cant update facebook texting it might cr...,0
1,kenichan dived many times ball managed save r...,0
2,whole body feels itchy like fire,0
3,nationwideclass no behaving all im mad here ca...,0
4,kwesidei whole crew,0
...,...,...
1599994,woke up school best feeling ever,4
1599995,thewdbcom cool hear old walt interviews â« h...,4
1599996,ready mojo makeover ask details,4
1599997,happy th birthday boo alll time tupac amaru sh...,4


In [ ]:
tokenizer=RegexpTokenizer(r'\w+')

In [ ]:
dff['Tweet']=dff['Tweet'].apply(tokenizer.tokenize)

In [ ]:
st=PorterStemmer()

In [ ]:
def stemming(text):
  stemmed_words=[st.stem(word) for word in text]
  return ' '.join(stemmed_words)

In [ ]:
dff['Tweet']=dff['Tweet'].apply(stemming)

In [ ]:
lm=WordNetLemmatizer()

In [ ]:
def lemma(text):
  words=text.split()
  lemma_words=[lm.lemmatize(word) for word in words]
  return ' '.join(lemma_words)

In [ ]:
dff['Tweet']=dff['Tweet'].apply(lemma)

In [ ]:
X=dff['Tweet']
y=dff['Polarity']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
vec=TfidfVectorizer(ngram_range=(1,2), max_features=500000)

In [ ]:
vec.fit(x_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [ ]:
x_train=vec.transform(x_train)
x_test=vec.transform(x_test)

In [ ]:
model=LogisticRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
print('Accuracy score:',accuracy_score(y_test,y_pred))

Accuracy score: 0.793646875


In [ ]:
print(cf_matrix)

[[123972  35522]
 [ 30511 129995]]
